# MARATONA BEHIND THE CODE 2020

## DESAFIO 6 - LIT

<hr>

## Download dos conjuntos de dados em formato .csv

In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_columns', 25)

In [3]:
dataset = pd.read_csv('training_dataset.csv')
dataset.tail()

id     graduacao                universidade  \
15852  620397030.0   Bacharelado                         UFF   
15853  229931283.0   Bacharelado                         NaN   
15854  480674907.0     Tecnólogo                        UNIP   
15855  559626807.0  SEM FORMAÇÃO  UNIVERSIDADE NOVE DE JULHO   
15856  743652801.0           NaN                      FGV-RJ   

                     profissao  organizacao  pretende_fazer_cursos_lit  \
15852                   Outros     Borracha                        0.0   
15853                 Advogado          NaN                        0.0   
15854  Sócio/Dono/Proprietário          NaN                        0.0   
15855                 Advogado       Estado                        0.0   
15856                      NaN  Siderurgica                        1.0   

       interesse_mba_lit  importante_ter_certificado  horas_semanais_estudo  \
15852                0.0                         1.0                    8.0   
15853                0.0                         1.0                    7.0   
15854                NaN                         1.0                    7.0   
15855                0.0                         1.0                   10.0   
15856                1.0                         1.0                    9.0   

      como_conheceu_lit  total_modulos  modulos_iniciados  \
15852               NaN           10.0                NaN   
15853          Linkedin           42.0               17.0   
15854            Outros           30.0                9.0   
15855               NaN          226.0              102.0   
15856            Outros          125.0               98.0   

       modulos_finalizados  certificados categoria  
15852                  NaN           NaN   perfil6  
15853                 15.0           NaN   perfil5  
15854                  8.0           0.0   perfil5  
15855                 93.0           1.0   perfil1  
15856                 97.0           1.0   perfil1

Sobre o arquivo "training_dataset.csv", temos algumas informações gerais sobre os usuários da plataforma:

**id**

**graduacao**

**universidade**

**profissao**

**organizacao**

**pretende_fazer_cursos_lit**

**interesse_mba_lit**

**importante_ter_certificado**

**horas_semanais_estudo**

**como_conheceu_lit**

**total_modulos**

**modulos_iniciados**

**modulos_finalizados**

**certificados**

**categoria**

In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15857 entries, 0 to 15856
Data columns (total 15 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          13945 non-null  float64
 1   graduacao                   13950 non-null  object 
 2   universidade                13920 non-null  object 
 3   profissao                   13977 non-null  object 
 4   organizacao                 13961 non-null  object 
 5   pretende_fazer_cursos_lit   13989 non-null  float64
 6   interesse_mba_lit           14003 non-null  float64
 7   importante_ter_certificado  13918 non-null  float64
 8   horas_semanais_estudo       13959 non-null  float64
 9   como_conheceu_lit           13915 non-null  object 
 10  total_modulos               13987 non-null  float64
 11  modulos_iniciados           14044 non-null  float64
 12  modulos_finalizados         13924 non-null  float64
 13  certificados                139

In [5]:
dataset.nunique()

id                            13945
graduacao                         6
universidade                     21
profissao                        12
organizacao                      11
pretende_fazer_cursos_lit         2
interesse_mba_lit                 2
importante_ter_certificado        1
horas_semanais_estudo             9
como_conheceu_lit                 9
total_modulos                   578
modulos_iniciados               372
modulos_finalizados             339
certificados                     23
categoria                         6
dtype: int64

<hr>

## Detalhamento do desafio: classificação multiclasse

Este é um desafio cujo objetivo de negócio é a segmentação dos usuários de uma plataforma de ensino. Para tal, podemos utilizar duas abordagens: aprendizado de máquina supervisionado (classificação) ou não-supervisionado (clustering). Neste desafio será aplicada a classificação, pois é disponível um dataset já com "labels", ou em outras palavras, já com exemplos de dados juntamente com a variável alvo.

Na biblioteca scikit-learn temos diversos algoritmos para classificação. O participante é livre para utilizar o framework que desejar para completar esse desafio.

Neste notebook será mostrado um exeplo de uso do algoritmo "Decision Tree" para classificar parte dos estudantes em seis diferentes perfís.

# Divisão do banco

A coluna-alvo neste desafio é a coluna ``categoria``

<hr>

In [6]:

from sklearn.model_selection import StratifiedShuffleSplit

In [7]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

In [8]:
for train_index, val_index in split.split(dataset, dataset.categoria):
    data_train = dataset.loc[train_index]    
    data_val = dataset.loc[val_index]

In [9]:
print(data_train.shape)
print(data_val.shape)

(12685, 15)
(3172, 15)


# Pre-processando o dataset antes do treinamento

### Removendo todas as linhas que possuem algum valor nulos em determinadas colunas

Usando o método Pandas **DataFrame.dropna()** você pode remover todas as linhas nulas do dataset.

Docs: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html

In [10]:
dd = data_train.dropna()

In [11]:
x_train = dd.drop(['id', 'importante_ter_certificado', 'categoria'], axis=1)

In [12]:
y_train = dd.categoria

In [13]:
x_train['rate1'] = x_train.modulos_finalizados/x_train.modulos_iniciados
x_train['rate2'] = x_train.certificados /x_train.modulos_iniciados
x_train['rate3'] = x_train.modulos_finalizados / x_train.total_modulos
x_train['rate4'] = x_train.certificados/x_train.total_modulos
x_train['rate5'] = x_train.modulos_iniciados/x_train.total_modulos
x_train['rate6'] = x_train.certificados /x_train.modulos_finalizados
x_train['rate7'] = x_train.certificados /x_train.total_modulos

x_train['mul1'] = x_train.certificados * x_train.modulos_finalizados

x_train['add1'] = x_train.certificados + x_train.modulos_finalizados

In [14]:
x_train.tail()

graduacao                universidade                profissao  \
9332     Tecnólogo  Escola Paulista de Direito          Analista Senior   
2945     Tecnólogo  UNIVERSIDADE NOVE DE JULHO  Sócio/Dono/Proprietário   
5659     Tecnólogo                         UFF               Supervisor   
1195  SEM FORMAÇÃO                         USP                 Analista   
1863  Licenciatura  UNIVERSIDADE NOVE DE JULHO                  Diretor   

         organizacao  pretende_fazer_cursos_lit  interesse_mba_lit  \
9332       Laminados                        0.0                0.0   
2945      e-commerce                        1.0                1.0   
5659  Entretenimento                        0.0                0.0   
1195      e-commerce                        1.0                1.0   
1863  Entretenimento                        1.0                1.0   

      horas_semanais_estudo                  como_conheceu_lit  total_modulos  \
9332                    7.0         Mídia (revista/jornal/web)          100.0   
2945                   10.0  Minha empresa - benefício LITpass           29.0   
5659                    4.0         Mídia (revista/jornal/web)           31.0   
1195                    7.0                             Outros           33.0   
1863                   10.0                         Saint Paul           37.0   

      modulos_iniciados  modulos_finalizados  certificados     rate1  \
9332               99.0                 99.0           3.0  1.000000   
2945                5.0                  3.0           0.0  0.600000   
5659               11.0                 10.0           0.0  0.909091   
1195                4.0                  2.0           0.0  0.500000   
1863                4.0                  1.0           0.0  0.250000   

         rate2     rate3  rate4     rate5     rate6  rate7   mul1   add1  
9332  0.030303  0.990000   0.03  0.990000  0.030303   0.03  297.0  102.0  
2945  0.000000  0.103448   0.00  0.172414  0.000000   0.00    0.0    3.0  
5659  0.000000  0.322581   0.00  0.354839  0.000000   0.00    0.0   10.0  
1195  0.000000  0.060606   0.00  0.121212  0.000000   0.00    0.0    2.0  
1863  0.000000  0.027027   0.00  0.108108  0.000000   0.00    0.0    1.0

In [15]:
from sklearn.preprocessing import OrdinalEncoder

In [16]:
cat_cols = list(x_train.select_dtypes(include=['object']).columns)
cat_cols

['graduacao', 'universidade', 'profissao', 'organizacao', 'como_conheceu_lit']

In [17]:
ordinal = OrdinalEncoder()

In [18]:
xx = x_train.copy()

In [19]:
aa = pd.DataFrame( ordinal.fit_transform(x_train[cat_cols]))
aa.index = x_train.index

xx[cat_cols] = aa

In [20]:
xx.tail()

graduacao  universidade  profissao  organizacao  \
9332        5.0           1.0        2.0          5.0   
2945        5.0          16.0       11.0          9.0   
5659        5.0           7.0       10.0          3.0   
1195        4.0          18.0        1.0          9.0   
1863        2.0          16.0        5.0          3.0   

      pretende_fazer_cursos_lit  interesse_mba_lit  horas_semanais_estudo  \
9332                        0.0                0.0                    7.0   
2945                        1.0                1.0                   10.0   
5659                        0.0                0.0                    4.0   
1195                        1.0                1.0                    7.0   
1863                        1.0                1.0                   10.0   

      como_conheceu_lit  total_modulos  modulos_iniciados  \
9332                5.0          100.0               99.0   
2945                4.0           29.0                5.0   
5659                5.0           31.0               11.0   
1195                6.0           33.0                4.0   
1863                7.0           37.0                4.0   

      modulos_finalizados  certificados     rate1     rate2     rate3  rate4  \
9332                 99.0           3.0  1.000000  0.030303  0.990000   0.03   
2945                  3.0           0.0  0.600000  0.000000  0.103448   0.00   
5659                 10.0           0.0  0.909091  0.000000  0.322581   0.00   
1195                  2.0           0.0  0.500000  0.000000  0.060606   0.00   
1863                  1.0           0.0  0.250000  0.000000  0.027027   0.00   

         rate5     rate6  rate7   mul1   add1  
9332  0.990000  0.030303   0.03  297.0  102.0  
2945  0.172414  0.000000   0.00    0.0    3.0  
5659  0.354839  0.000000   0.00    0.0   10.0  
1195  0.121212  0.000000   0.00    0.0    2.0  
1863  0.108108  0.000000   0.00    0.0    1.0

In [21]:
x_train = xx

### Scaler

In [22]:
from sklearn.preprocessing import StandardScaler

In [23]:
scaler = StandardScaler()

In [24]:
#x_train = pd.DataFrame(scaler.fit_transform(x_train), columns=x_train.columns)

## Repetindo no banco de validação

In [25]:
dd = data_val.dropna()
x_val = dd.drop(['id', 'importante_ter_certificado', 'categoria'], axis=1)
y_val = dd.categoria

In [26]:
x_val['rate1'] = x_val.modulos_finalizados/x_val.modulos_iniciados
x_val['rate2'] = x_val.certificados /x_val.modulos_iniciados
x_val['rate3'] = x_val.modulos_finalizados / x_val.total_modulos
x_val['rate4'] = x_val.certificados/x_val.total_modulos
x_val['rate5'] = x_val.modulos_iniciados/x_val.total_modulos
x_val['rate6'] = x_val.certificados /x_val.modulos_finalizados
x_val['rate7'] = x_val.certificados /x_val.total_modulos

x_val['mul1'] = x_val.certificados * x_val.modulos_finalizados

x_val['add1'] = x_val.certificados + x_val.modulos_finalizados

In [27]:
aa = pd.DataFrame( ordinal.transform(x_val[cat_cols]))
aa.index = x_val.index

x_val[cat_cols] = aa

In [28]:
x_val.head()

graduacao  universidade  profissao  organizacao  \
15745        3.0           9.0        9.0          7.0   
9322         1.0          11.0        4.0          7.0   
7429         1.0          13.0        6.0          3.0   
7790         1.0           3.0        5.0          7.0   
5696         2.0           4.0        0.0          1.0   

       pretende_fazer_cursos_lit  interesse_mba_lit  horas_semanais_estudo  \
15745                        1.0                1.0                    7.0   
9322                         1.0                1.0                    9.0   
7429                         1.0                1.0                    7.0   
7790                         1.0                1.0                   10.0   
5696                         1.0                1.0                    7.0   

       como_conheceu_lit  total_modulos  modulos_iniciados  \
15745                1.0           31.0                8.0   
9322                 2.0          135.0               47.0   
7429                 3.0           33.0                3.0   
7790                 2.0          239.0               42.0   
5696                 4.0           57.0               11.0   

       modulos_finalizados  certificados     rate1     rate2     rate3  \
15745                  5.0           0.0  0.625000  0.000000  0.161290   
9322                  39.0           1.0  0.829787  0.021277  0.288889   
7429                   1.0           0.0  0.333333  0.000000  0.030303   
7790                  25.0           0.0  0.595238  0.000000  0.104603   
5696                   7.0           0.0  0.636364  0.000000  0.122807   

          rate4     rate5     rate6     rate7  mul1  add1  
15745  0.000000  0.258065  0.000000  0.000000   0.0   5.0  
9322   0.007407  0.348148  0.025641  0.007407  39.0  40.0  
7429   0.000000  0.090909  0.000000  0.000000   0.0   1.0  
7790   0.000000  0.175732  0.000000  0.000000   0.0  25.0  
5696   0.000000  0.192982  0.000000  0.000000   0.0   7.0

In [29]:
#x_val = pd.DataFrame(scaler.transform(x_val), columns=x_val.columns)

# Treinamento

In [30]:
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

In [41]:
clf = RandomForestClassifier(max_depth=10)

In [42]:
clf.fit(x_train, y_train)

pred_train = clf.predict(x_train)

In [34]:
print(classification_report(pred_train, y_train))

              precision    recall  f1-score   support

     perfil1       0.85      0.94      0.89       310
     perfil2       0.89      0.86      0.87       591
     perfil3       0.83      0.87      0.85       125
     perfil4       0.89      0.85      0.87       248
     perfil5       0.83      0.85      0.84       245
     perfil6       0.90      0.87      0.88       614

    accuracy                           0.87      2133
   macro avg       0.86      0.87      0.87      2133
weighted avg       0.87      0.87      0.87      2133



### validação

In [35]:
pred_val = clf.predict(x_val)

In [36]:

print(classification_report(y_val, pred_val))

              precision    recall  f1-score   support

     perfil1       0.82      0.87      0.84        75
     perfil2       0.81      0.82      0.81       151
     perfil3       0.84      0.76      0.80        42
     perfil4       0.85      0.81      0.83        69
     perfil5       0.88      0.89      0.88        79
     perfil6       0.86      0.85      0.86       142

    accuracy                           0.84       558
   macro avg       0.84      0.83      0.84       558
weighted avg       0.84      0.84      0.84       558



In [37]:
print(accuracy_score(y_val, pred_val))

0.8387096774193549


## Insights
SelectKBest

In [38]:

from sklearn.feature_selection import SelectKBest

In [39]:

for i in range(x_train.shape[1], 5, -1):
    skb = SelectKBest(k = i)
    xx = skb.fit_transform(x_train, y_train)
    clf.fit(xx, y_train)
    pred = clf.predict(skb.transform(x_val))
    print(i, accuracy_score(y_val, pred))

21 0.8369175627240143
20 0.8369175627240143
19 0.8369175627240143
18 0.8351254480286738
17 0.8333333333333334
16 0.8315412186379928
15 0.8315412186379928
14 0.8279569892473119
13 0.8225806451612904
12 0.8279569892473119
11 0.8189964157706093
10 0.8261648745519713
9 0.8225806451612904
8 0.8243727598566308
7 0.8225806451612904
6 0.589605734767025


In [40]:
for i in range(1,30, 2):
    clf2 = RandomForestClassifier(max_depth=i)
    clf2.fit(x_train, y_train)
    pred = clf2.predict(x_val)
    print(i, accuracy_score(y_val, pred))

1 0.5734767025089605
3 0.7544802867383512
5 0.8189964157706093
7 0.8387096774193549
9 0.8387096774193549
11 0.8387096774193549
13 0.8387096774193549
15 0.8369175627240143
17 0.8369175627240143
19 0.8297491039426523
21 0.8351254480286738
23 0.8351254480286738
25 0.8369175627240143
27 0.8387096774193549
29 0.8369175627240143


### Processando valores NaN com o SimpleImputer do sklearn

Para os valores NaN, usaremos a substituição pela constante 0 como **exemplo**.

Você pode escolher a estratégia que achar melhor para tratar os valores nulos :)

Docs: https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html?highlight=simpleimputer#sklearn.impute.SimpleImputer

In [ ]:
from sklearn.impute import SimpleImputer
import numpy as np


impute_zeros = SimpleImputer(
    missing_values=np.nan,
    strategy='constant',
    fill_value=0,
    verbose=0,
    copy=True
)

In [ ]:
# Exibindo os dados ausentes do conjunto de dados antes da primeira transformação (df)
print("Valores nulos no df_training_dataset antes da transformação SimpleImputer: \n\n{}\n".format(df_training_dataset.isnull().sum(axis = 0)))

# Aplicando a transformação ``SimpleImputer`` no conjunto de dados base
impute_zeros.fit(X=df_training_dataset)

# Reconstruindo um Pandas DataFrame com os resultados
df_training_dataset_imputed = pd.DataFrame.from_records(
    data=impute_zeros.transform(
        X=df_training_dataset
    ),
    columns=df_training_dataset.columns
)

# Exibindo os dados ausentes do conjunto de dados após a primeira transformação (df)
print("Valores nulos no df_training_dataset após a transformação SimpleImputer: \n\n{}\n".format(df_training_dataset_imputed.isnull().sum(axis = 0)))

### Eliminando colunas indesejadas

Vamos **demonstrar** abaixo como usar o método **DataFrame.drop()**.

Docs: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html

In [ ]:
df_training_dataset_imputed.tail()

In [ ]:
df_training_dataset_rmcolumns = df_training_dataset_imputed.drop(columns=['id', 'graduacao', 'universidade', 'organizacao', 'como_conheceu_lit'], inplace=False)

In [ ]:
df_training_dataset_rmcolumns.tail()

# Atenção!

As colunas removidas acima são apenas para fim de exemplo, você pode usar as colunas que quiser e inclusive criar novas colunas com dados que achar importantes!


### Tratamento de de variáveis categóricas

Como mencionado antes, os computadores não são bons com variáveis "categóricas" (ou strings).

Dado uma coluna com variável categórica, o que podemos realizar é a codificação dessa coluna em múltiplas colunas contendo variáveis binárias. Esse processo é chamado de "one-hot-encoding" ou "dummy encoding". Se você não é familiarizado com esses termos, você pode pesquisar mais sobre isso na internet :)

In [ ]:
# Tratando variáveis categóricas com o método Pandas ``get_dummies()''
df_training = pd.get_dummies(df_training_dataset_rmcolumns, columns=['profissao'])
df_training.tail()

# Atenção!

A coluna **categoria** deve ser mantida como uma string. Você não precisa processar/codificar a variável-alvo.

<hr>

## Treinando um classificador com base em uma árvore de decisão

### Selecionando FEATURES e definindo a variável TARGET

In [ ]:
df_training.columns

In [ ]:
features = df_training[
    [
        'pretende_fazer_cursos_lit', 'interesse_mba_lit',
        'importante_ter_certificado', 'horas_semanais_estudo', 'total_modulos',
        'modulos_iniciados', 'modulos_finalizados', 'certificados',
        'profissao_0', 'profissao_Advogado', 'profissao_Analista',
        'profissao_Analista Senior', 'profissao_Assessor',
        'profissao_Coordenador', 'profissao_Diretor', 'profissao_Engenheiro',
        'profissao_Gerente', 'profissao_Outros', 'profissao_SEM EXPERIÊNCIA',
        'profissao_Supervisor', 'profissao_Sócio/Dono/Proprietário'
    ]
]
target = df_training['categoria']  ## NÃO TROQUE O NOME DA VARIÁVEL TARGET.

### Dividindo nosso conjunto de dados em conjuntos de treinamento e teste

In [ ]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=133)

### Treinando uma árvore de decisão

In [ ]:
# Método para creacion de modelos basados en arbol de desición
from sklearn.tree import DecisionTreeClassifier


dtc = DecisionTreeClassifier(max_depth=15).fit(X_train, y_train)

### Fazendo previsões na amostra de teste

In [ ]:
y_pred = dtc.predict(X_test)
print(y_pred)

### Analisando a qualidade do modelo através da matriz de confusão

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import itertools


def plot_confusion_matrix(cm, target_names, title='Confusion matrix', cmap=None, normalize=True):
    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy
    if cmap is None:
        cmap = plt.get_cmap('Blues')
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix


plot_confusion_matrix(confusion_matrix(y_test, y_pred), ['parfil1', 'perfil2', 'perfil3', 'perfil4', 'perfil5', 'perfil6'])

<hr>

## Scoring dos dados necessários para entregar a solução

Como entrega da sua solução, esperamos os resultados classificados no seguinte dataset chamado "to_be_scored.csv":

### Download da "folha de respostas"

In [ ]:
!wget --no-check-certificate --content-disposition https://raw.githubusercontent.com/vanderlei-test/dataset-3/master/to_be_scored.csv
df_to_be_scored = pd.read_csv(r'to_be_scored.csv')
df_to_be_scored.tail()

# Atenção!

O dataframe ``to_be_scored`` é a sua "folha de respostas". Note que a coluna "categoria" não existe nessa amostra, que não pode ser então utilizada para treino de modelos de aprendizado supervisionado.

In [ ]:
df_to_be_scored.info()

<hr>

# Atenção!

# Para poder aplicar seu modelo e classificar a folha de respostas, você precisa primeiro aplicar as mesmas transformações com colunas que você aplicou no dataset de treino.

# Não remova ou adicione linhas na folha de respostas. 

# Não altere a ordem das linhas na folha de respostas.

# Ao final, as 1000 entradas devem estar classificadas, com os valores previstos em uma coluna chamada "target"

<hr>

Na célula abaixo, repetimos rapidamente os mesmos passos de pré-processamento usados no exemplo dado com árvore de decisão

In [ ]:
# 1 - Removendo linhas com valores NaN em "certificados" e "total_modulos"
df_to_be_scored_1 = df_to_be_scored.dropna(axis='index', how='any', subset=['certificados', 'total_modulos'])

# 2 - Inputando zeros nos valores faltantes
impute_zeros.fit(X=df_to_be_scored_1)
df_to_be_scored_2 = pd.DataFrame.from_records(
    data=impute_zeros.transform(
        X=df_to_be_scored_1
    ),
    columns=df_to_be_scored_1.columns
)

# 3 - Remoção de colunas
df_to_be_scored_3 = df_to_be_scored_2.drop(columns=['id', 'graduacao', 'universidade', 'organizacao', 'como_conheceu_lit'], inplace=False)

# 4 - Encoding com "dummy variables"
df_to_be_scored_4 = pd.get_dummies(df_to_be_scored_3, columns=['profissao'])

df_to_be_scored_4.tail()

<hr>

Pode ser verificado abaixo que as colunas da folha de resposta agora são idênticas às que foram usadas para treinar o modelo:

In [ ]:
df_training[
    [
        'pretende_fazer_cursos_lit', 'interesse_mba_lit',
        'importante_ter_certificado', 'horas_semanais_estudo', 'total_modulos',
        'modulos_iniciados', 'modulos_finalizados', 'certificados',
        'profissao_0', 'profissao_Advogado', 'profissao_Analista',
        'profissao_Analista Senior', 'profissao_Assessor',
        'profissao_Coordenador', 'profissao_Diretor', 'profissao_Engenheiro',
        'profissao_Gerente', 'profissao_Outros', 'profissao_SEM EXPERIÊNCIA',
        'profissao_Supervisor', 'profissao_Sócio/Dono/Proprietário'
    ]
].columns

In [ ]:
df_to_be_scored_4.columns

# Atenção

Para todas colunas que não existirem no "df_to_be_scored", você pode usar a técnica abaixo para adicioná-las:

In [ ]:
df_to_be_scored_4['profissao_0'] = 0

In [ ]:
y_pred = dtc.predict(df_to_be_scored_4)
df_to_be_scored_4['target'] = y_pred
df_to_be_scored_4.tail()

### Salvando a folha de respostas como um arquivo .csv para ser submetido

In [ ]:
project.save_data(file_name="results.csv", data=df_to_be_scored_4.to_csv(index=False))

# Atenção

# A execução da célula acima irá criar um novo "data asset" no seu projeto no Watson Studio. Você precisará realizar o download deste arquivo juntamente com este notebook e criar um arquivo zip com os arquivos **results.csv** e **notebook.ipynb** para submissão. (os arquivos devem estar nomeados desta forma)

<hr>

## Parabéns!

Se você já está satisfeito com a sua solução, vá até a página abaixo e envie os arquivos necessários para submissão.

# https://lit.maratona.dev
